<a href="https://colab.research.google.com/github/UserLeeIsTired/simpleModel/blob/main/Regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic data reading


In [ ]:
import pandas as pd


#read file
data = pd.read_excel('/data.xlsx')


# filter out row with any column with NaN
data = data.dropna()

data = data.drop('Form\nFactor', axis=1)



Find unique names in column battery and chemistry

In [ ]:
# find unique name in battery and chemistry
battery_unique = data.Battery.unique()
chemistry_unique = data.Chemistry.unique()

print(battery_unique)
print(chemistry_unique)

['Panasonic 3L MIC' 'CATL 6C' 'CATL 6L' 'LG Chem 5L' 'LG Chem 5C'
 'Panasonic 3' 'Panasonic 1C' 'Panasonic 1L' 'Panasonic 3L' 'Panasonic 3C'
 'Panasonic A3']
['NCA MIC' 'LFP' 'NCM' 'NCA']


apply one hot


In [ ]:
import sklearn
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
ohe_transform = ohe.fit_transform(data[['Battery', 'Chemistry']])


Merge the table and drop battery and chemistry


In [ ]:
data = pd.concat([data.drop(['Battery', 'Chemistry'], axis=1), ohe_transform], axis=1)


time to separate the result by y

In [ ]:
# Assume if degradation >= 0.08, it is a good time to change battery
data['Degradation'] = [1 if value >= 0.08 else 0 for value in data['Degradation']]
y = data['Degradation']
data = data.drop('Degradation', axis=1)

Before training the data, we have to normalize the data, to prevent uneven scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

data.columns = data.columns.astype(str)

data = scaler.fit_transform(data)


In [ ]:
print(data)

[[ 1.00147482 -0.41894827 -0.67157817 ... -0.50845272  2.76529441
  -0.68385363]
 [-1.57768667 -0.15651147 -0.6154385  ... -0.50845272 -0.36162515
  -0.68385363]
 [-1.00453967 -0.43276074 -0.50479808 ... -0.50845272 -0.36162515
  -0.68385363]
 ...
 [ 0.7544287  -0.25319871 -0.02093028 ... -0.50845272 -0.36162515
   1.46230123]
 [-1.00453967 -0.36369842 -0.14315419 ... -0.50845272 -0.36162515
  -0.68385363]
 [ 0.7544287   0.45123693 -0.25590575 ... -0.50845272 -0.36162515
   1.46230123]]


split validation and testing set

In [ ]:
from sklearn.model_selection import train_test_split

# split data into train and test, use a random state seed to make sure the result is constant every time
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=.3, random_state=1)

train the model

In [ ]:
from sklearn.linear_model import LogisticRegression
log_r = LogisticRegression()

# train model based on the data
log_r.fit(X_train, y_train)



LogisticRegression()

test the model

In [ ]:
y_pred = log_r.predict(X_test)

In [ ]:
y_pred

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,

Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9919137466307277
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       477
           1       0.99      0.98      0.99       265

    accuracy                           0.99       742
   macro avg       0.99      0.99      0.99       742
weighted avg       0.99      0.99      0.99       742

